<a href="https://colab.research.google.com/github/kevinkemps/hw3_image_classification/blob/main/hw_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Part a:
This section is just setting up a base model using keras (code based on sample keras notebook code). You can skip to section a.i for the parameter tuning

In [3]:
from keras.datasets import cifar10
from sklearn.model_selection import train_test_split
import math
import pandas as pd
import keras
from keras import models, layers, optimizers, regularizers
import numpy as np
import random
from sklearn import model_selection, preprocessing
import tensorflow as tf
from tqdm import tqdm
import matplotlib.pyplot as plt
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout
from skimage.color import rgb2gray
from keras.utils import to_categorical
import time


In [5]:
# load the dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# convert to grayscale images
X_train = rgb2gray(X_train)
X_test = rgb2gray(X_test)

# Preprocessing: Normalize the images.
train_images = (X_train / 255)
test_images = (X_test / 255)

# prompt: split the training data and labels into 1/5 validation and 4/5 training at random
train_images, val_images, train_labels, val_labels = train_test_split(train_images, y_train, test_size=1/5, random_state=42)

In [13]:
val_labels

array([[7],
       [8],
       [0],
       ...,
       [8],
       [2],
       [6]], dtype=uint8)

In [3]:
model = Sequential([Dense(64, input_shape=(1024,), activation='relu'),
                    Dense(32, activation='relu'),
                    Dense(10, activation='softmax')])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                65600     
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 10)                330       
                                                                 
Total params: 68010 (265.66 KB)
Trainable params: 68010 (265.66 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [4]:
# Compile model
model.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'],)

In [5]:
#flatten all groups of images
flatten_train_images = train_images.reshape((-1, 32*32))
flatten_val_images = val_images.reshape((-1, 32*32))
flatten_test_images = test_images.reshape((-1, 32*32))

In [6]:
print("Train image shape: ", train_images.shape, "Flattened image shape: ", flatten_train_images.shape)

Train image shape:  (40000, 32, 32) Flattened image shape:  (40000, 1024)


In [7]:
model.fit(flatten_train_images, to_categorical(train_labels), epochs=10, batch_size=256,)


Epoch 1/10
157/157 [==============================] - 4s 7ms/step - loss: 2.3023 - accuracy: 0.1052
Epoch 2/10
157/157 [==============================] - 1s 7ms/step - loss: 2.3021 - accuracy: 0.1059
Epoch 3/10
157/157 [==============================] - 1s 4ms/step - loss: 2.3021 - accuracy: 0.1072
Epoch 4/10
157/157 [==============================] - 1s 3ms/step - loss: 2.3021 - accuracy: 0.1174
Epoch 5/10
157/157 [==============================] - 1s 3ms/step - loss: 2.3020 - accuracy: 0.1096
Epoch 6/10
157/157 [==============================] - 1s 3ms/step - loss: 2.3020 - accuracy: 0.1017
Epoch 7/10
157/157 [==============================] - 1s 3ms/step - loss: 2.3020 - accuracy: 0.1020
Epoch 8/10
157/157 [==============================] - 1s 4ms/step - loss: 2.3020 - accuracy: 0.1032
Epoch 9/10
157/157 [==============================] - 1s 4ms/step - loss: 2.3020 - accuracy: 0.1017
Epoch 10/10
157/157 [==============================] - 1s 5ms/step - loss: 2.3020 - accuracy: 0.1017

In [8]:

# Evaluate your model's performance on the test data
performance = model.evaluate(flatten_test_images, to_categorical(y_test))
print("Accuracy on Test samples: {0}".format(performance[1]))

313/313 [==============================] - 1s 3ms/step - loss: 2.3020 - accuracy: 0.1000
Accuracy on Test samples: 0.10000000149011612


(a.i) Hyper Parameter tuning

In [14]:
'''
Helper functions to build, train, and test the model.

'''
def build_model(nodes=[128, 64, 10], activation='relu'):
  layer_list = []

  for layer in range(len(nodes)):
      if type(activation) == list:
        activation = activation[layer]
      if layer == 0:
        layer_list.append(Dense(nodes[layer], input_shape=(1024,), activation=activation))
      else:
        layer_list.append(Dense(nodes[layer], activation=activation))
  model = Sequential(layer_list)
  model.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'],)
  print(model.summary())
  return model


def train_model(model, train_X, train_labels, epochs=10, batch_size=256):
  start = time.time()
  model.fit(flatten_train_images, to_categorical(train_labels), epochs=10, batch_size=256,)
  print(f"The model took {time.time() - start} seconds to train")


def test_model(model, test_X, test_labels):
  # Evaluate your model's performance on the test data
  performance = model.evaluate(test_X, to_categorical(test_labels))
  print("Accuracy on Test/Validation samples: {0}".format(performance[1]))

Experiment on the validation set with different FNN hyper-parameters, e.g.
layers, nodes per layer, activation function, dropout, weight regularization, etc.

In [15]:
model = build_model()
train_model(model=model, train_X=flatten_train_images, train_labels=train_labels)
test_model(model=model, test_X=flatten_val_images, test_labels=val_labels)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 128)               131200    
                                                                 
 dense_11 (Dense)            (None, 64)                8256      
                                                                 
 dense_12 (Dense)            (None, 10)                650       
                                                                 
Total params: 140106 (547.29 KB)
Trainable params: 140106 (547.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/10
157/157 [==============================] - 1s 4ms/step - loss: 6.2996 - accuracy: 0.0996
Epoch 2/10
157/157 [==============================] - 1s 4ms/step - loss: 6.2312 - accuracy: 0.0996
Epoch 3/10
157/157 [==============================] - 1s 4ms/step - loss:

In [16]:
model = build_model(nodes=[500, 250, 125, 10], activation=['relu', 'relu', 'relu', 'relu'])
train_model(model=model, train_X=flatten_train_images, train_labels=train_labels)
test_model(model=model, test_X=flatten_val_images, test_labels=val_labels)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 500)               512500    
                                                                 
 dense_14 (Dense)            (None, 250)               125250    
                                                                 
 dense_15 (Dense)            (None, 125)               31375     
                                                                 
 dense_16 (Dense)            (None, 10)                1260      
                                                                 
Total params: 670385 (2.56 MB)
Trainable params: 670385 (2.56 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/10
157/157 [==============================] - 1s 4ms/step - loss: 7.6014 - accuracy: 0.0991
Epoch 2/10
157/157 [=========================

In [55]:
model = build_model(nodes=[800 ,500, 250, 125, 10], activation=['relu', 'relu', 'relu', 'relu', 'relu'])
train_model(model=model, train_X=flatten_train_images, train_labels=train_labels)
test_model(model=model, test_X=flatten_val_images, test_labels=val_labels)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_29 (Dense)            (None, 800)               820000    
                                                                 
 dense_30 (Dense)            (None, 500)               400500    
                                                                 
 dense_31 (Dense)            (None, 250)               125250    
                                                                 
 dense_32 (Dense)            (None, 125)               31375     
                                                                 
 dense_33 (Dense)            (None, 10)                1260      
                                                                 
Total params: 1378385 (5.26 MB)
Trainable params: 1378385 (5.26 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/10


In [18]:
model = build_model(nodes=[800, 700, 600, 500, 400, 300, 250, 125, 50, 25, 10])
train_model(model=model, train_X=flatten_train_images, train_labels=train_labels)
test_model(model=model, test_X=flatten_val_images, test_labels=val_labels)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 800)               820000    
                                                                 
 dense_18 (Dense)            (None, 700)               560700    
                                                                 
 dense_19 (Dense)            (None, 600)               420600    
                                                                 
 dense_20 (Dense)            (None, 500)               300500    
                                                                 
 dense_21 (Dense)            (None, 400)               200400    
                                                                 
 dense_22 (Dense)            (None, 300)               120300    
                                                                 
 dense_23 (Dense)            (None, 250)              